In [ ]:
!wget --no-check-certificate 'https://docs.google.com/uc?export=download&id=1hC3WvLgciFrHln_cGd9OEzyojNSkaTZB' -O kaggle.json

--2022-05-07 05:03:27--  https://docs.google.com/uc?export=download&id=1hC3WvLgciFrHln_cGd9OEzyojNSkaTZB
Resolving docs.google.com (docs.google.com)... 142.250.73.206, 2607:f8b0:4004:829::200e
Connecting to docs.google.com (docs.google.com)|142.250.73.206|:443... connected.
HTTP request sent, awaiting response... 303 See Other
Location: https://doc-0g-a4-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/0vlbk805hi6u047gif73kdtob8vhdm3b/1651899750000/05189201525835114386/*/1hC3WvLgciFrHln_cGd9OEzyojNSkaTZB?e=download [following]
--2022-05-07 05:03:27--  https://doc-0g-a4-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/0vlbk805hi6u047gif73kdtob8vhdm3b/1651899750000/05189201525835114386/*/1hC3WvLgciFrHln_cGd9OEzyojNSkaTZB?e=download
Resolving doc-0g-a4-docs.googleusercontent.com (doc-0g-a4-docs.googleusercontent.com)... 142.251.16.132, 2607:f8b0:4004:c17::84
Connecting to doc-0g-a4-docs.googleusercontent.com (doc-0g-a4-docs.googleuserconte

In [ ]:
!pip install -q kaggle
!mkdir ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json
!kaggle competitions download -c deepfake-detection-challenge

100% 4.12G/4.13G [00:40<00:00, 191MB/s]
100% 4.13G/4.13G [00:40<00:00, 110MB/s]


In [ ]:
!unzip deepfake-detection-challenge.zip

Archive:  deepfake-detection-challenge.zip
  inflating: sample_submission.csv   
  inflating: test_videos/aassnaulhq.mp4  
  inflating: test_videos/aayfryxljh.mp4  
  inflating: test_videos/acazlolrpz.mp4  
  inflating: test_videos/adohdulfwb.mp4  
  inflating: test_videos/ahjnxtiamx.mp4  
  inflating: test_videos/ajiyrjfyzp.mp4  
  inflating: test_videos/aktnlyqpah.mp4  
  inflating: test_videos/alrtntfxtd.mp4  
  inflating: test_videos/aomqqjipcp.mp4  
  inflating: test_videos/apedduehoy.mp4  
  inflating: test_videos/apvzjkvnwn.mp4  
  inflating: test_videos/aqrsylrzgi.mp4  
  inflating: test_videos/axfhbpkdlc.mp4  
  inflating: test_videos/ayipraspbn.mp4  
  inflating: test_videos/bcbqxhziqz.mp4  
  inflating: test_videos/bcvheslzrq.mp4  
  inflating: test_videos/bdshuoldwx.mp4  
  inflating: test_videos/bfdopzvxbi.mp4  
  inflating: test_videos/bfjsthfhbd.mp4  
  inflating: test_videos/bjyaxvggle.mp4  
  inflating: test_videos/bkcyglmfci.mp4  
  inflating: test_videos/bktkwbcawi.m

In [ ]:
!mkdir train_sample_audios

In [ ]:
import json
import os
import shutil
import cv2
import numpy as np
import subprocess
import tensorflow as tf
from tensorflow import keras
import soundfile
from pathlib import Path
import matplotlib.pyplot as plt
%matplotlib inline

metadata_file = "train_sample_videos/metadata.json"
input_path = "train_sample_videos/"
input_audio_path = "train_sample_audios/"
frame_save_path = "train_video/frames/"

STRIDE = 0.1
AUD_SAMPLING_RATE = 22050
MAX_IMAGE_SIZE = 1024

def get_frames_from_video(video_file, stride=1.0):
    """
    video_file - path to file
    stride - i.e 1.0 - extract frame every second, 0.5 - extract every 0.5 seconds
    return: list of images, list of frame times in seconds
    """
    video = cv2.VideoCapture(video_file)
    fps = video.get(cv2.CAP_PROP_FPS)
    i = 0.
    images = []
    frame_times = []

    while video.isOpened():
        ret, frame = video.read()
        if ret and len(images) < 30:
            images.append(frame)
            frame_times.append(i)
            i += stride
            video.set(1, round(i * fps))
        else:
            video.release()
            break
    return images, frame_times


def convert_to_wav(vid_filename):
  command = f"ffmpeg -i {input_path}/{vid_filename} -vn -ar {AUD_SAMPLING_RATE} -ac 1 -acodec pcm_s16le {input_audio_path}/{vid_filename[:-4]}.wav"
  subprocess.call(command, shell=True)

with open(metadata_file) as json_file:
    metadata = json.load(json_file)

image_dataset = []
audio_dataset = []
labels = []

for i in range(3):
  key = list(metadata.keys())[i]
  lab = metadata[key]["label"]
  if(lab == "FAKE"):
    labels.append(0)
  else:
    labels.append(1)

  sample_video = os.path.join(input_path, key)
  sample_audio = os.path.join(input_audio_path, key[:-4]+'.wav')

  images, frame_times = get_frames_from_video(sample_video, STRIDE)
  convert_to_wav(key)
  audio = tf.io.read_file(sample_audio)
  audio, _ = tf.audio.decode_wav(audio, desired_channels=1, desired_samples=3*AUD_SAMPLING_RATE)

  image_dataset.append(images)
  audio_dataset.append(audio)
# images = [resize_if_necessary(image, MAX_IMAGE_SIZE) for image in images]

# Note : Image height : 1080 and width : 1920


In [ ]:
image_dataset = tf.convert_to_tensor(image_dataset)
image_dataset = np.array(image_dataset)
audio_dataset = tf.convert_to_tensor(audio_dataset)
audio_dataset = np.array(audio_dataset)

In [ ]:
print(image_dataset.shape)

(3, 30, 1080, 1920, 3)


In [ ]:
print(audio_dataset.shape)

(3, 66150, 1)


In [ ]:
labels = np.array(labels)

In [ ]:
!pip install tensorflow_addons

In [ ]:
from datetime import datetime
import os,cv2
#from cv2 import getRotationMatrix2D, warpAffine,getAffineTransform,resize,imread,BORDER_REFLECT
import numpy as np
#KERAS IMPORTS
from keras.applications.vgg16 import VGG16
from keras.callbacks import ProgbarLogger, EarlyStopping, ModelCheckpoint, TensorBoard
from keras.models import Model, Sequential
from keras.layers import Input, Convolution2D, MaxPooling2D, MaxPooling1D, Conv2DTranspose, Conv3D, Conv2D
from keras.layers import Add, Conv1D, concatenate, Dense, Conv1D, TimeDistributed, Flatten, Resizing
from keras.layers.core import Reshape, Activation, Dropout
import tensorflow_addons as tfa
from keras.preprocessing.image import *
from tensorflow.keras.optimizers import SGD
from keras.backend import expand_dims, shape
from tensorflow.keras.utils import plot_model
import matplotlib.pyplot as plt

input_shape = (30, 1080, 1920, 3)
input_rgb = Input(shape=input_shape)
input_img = TimeDistributed( Resizing(135, 240))(input_rgb)

conv11 = Conv3D(64, (3, 7, 7), padding="same", activation="relu", input_shape=input_shape[1:])(input_img)
pool11 = TimeDistributed( MaxPooling2D(pool_size=(4, 4)))(conv11)

conv12 = Conv3D(64, (3, 3, 3), padding="same", activation="relu", input_shape=input_shape[1:])(pool11)
pool12 = TimeDistributed( MaxPooling2D(pool_size=(4, 4)))(conv12)

conv13 = Conv3D(128, (3, 3, 3), padding="same", activation="relu", input_shape=input_shape[1:])(pool12)
pool13 = TimeDistributed( MaxPooling2D(pool_size=(2, 2)))(conv13)

conv14 = Conv3D(256, (3, 3, 3), padding="same", activation="relu", input_shape=input_shape[1:])(pool13)
pool14 = TimeDistributed( MaxPooling2D(pool_size=(2, 2)))(conv14)

conv15 = Conv3D(512, (3, 3, 3), padding="same", activation="relu", input_shape=input_shape[1:])(pool14)
pool15 = TimeDistributed( MaxPooling2D(pool_size=(2, 2), strides=(2, 4)))(conv15)

flat1 = Flatten()(pool15)
flat1 = keras.layers.BatchNormalization()(flat1)
flat1 = tfa.layers.AdaptiveAveragePooling1D(512)(flat1)

# second input model
input_stft = Input(shape=(3*AUD_SAMPLING_RATE, 1))

conv21 = Conv1D(128, kernel_size=3, strides=4, padding="same", activation='relu')(input_stft)
conv22 = Conv1D(128, kernel_size=3, strides=4, padding="same", activation='relu')(conv21)
conv23 = Conv1D(256, kernel_size=3, strides=4, padding="same", activation='relu')(conv22)
conv24 = Conv1D(512, kernel_size=3, strides=4, padding="same", activation='relu')(conv23)

flat2 = Flatten()(conv24)
flat2 = tfa.layers.AdaptiveAveragePooling1D(512)(flat2)

# merge input models
merge = Add()([flat1, flat2])
merge = keras.layers.Dropout(0.2)(merge)
output = Dense(1, activation='softmax') (merge)

model = Model(inputs=[input_rgb, input_stft], outputs=output)

# # summarize layers
print(model.summary())
# exit()

model.compile(optimizer=SGD(learning_rate=0.1, decay=1e-6, momentum=0.9, nesterov=True),
              loss='binary_crossentropy',
              metrics=['accuracy'])

# history = model.fit_generator(train_generator, steps_per_epoch=100, verbose=1, epochs=n_epochs, validation_data=validation_generator, validation_steps=10, use_multiprocessing=True)

# print(model.evaluate_generator(test_generator, steps=100))

# plot_model(model, to_file='model.png')

# Plot training & validation accuracy values
# plt.plot(history.history['acc'])
# plt.plot(history.history['val_acc'])
# plt.title('Model accuracy')
# plt.ylabel('Accuracy')
# plt.xlabel('Epoch')
# plt.legend(['Train', 'Test'], loc='upper left')
# plt.savefig("accuracy.png")

# # Plot training & validation loss values
# plt.plot(history.history['loss'])
# plt.plot(history.history['val_loss'])
# plt.title('Model loss')
# plt.ylabel('Loss')
# plt.xlabel('Epoch')
# plt.legend(['Train', 'Test'], loc='upper left')
# plt.savefig("loss.png")

# model.save('final_model')

Model: "model_2"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_5 (InputLayer)           [(None, 30, 1080, 1  0           []                               
                                920, 3)]                                                          
                                                                                                  
 time_distributed_12 (TimeDistr  (None, 30, 135, 240  0          ['input_5[0][0]']                
 ibuted)                        , 3)                                                              
                                                                                                  
 conv3d_10 (Conv3D)             (None, 30, 135, 240  28288       ['time_distributed_12[0][0]']    
                                , 64)                                                       

In [ ]:
history = model.fit([image_dataset, audio_dataset], labels, epochs = 10, verbose=1, use_multiprocessing=True)

Epoch 1/10
1/1 [==============================] - 25s 25s/step - loss: 0.6067 - accuracy: 0.3333
Epoch 2/10
1/1 [==============================] - 19s 19s/step - loss: 2.9729 - accuracy: 0.3333
Epoch 3/10
1/1 [==============================] - 20s 20s/step - loss: 0.0125 - accuracy: 0.3333
Epoch 4/10
1/1 [==============================] - 20s 20s/step - loss: 1.9417e-05 - accuracy: 0.3333
Epoch 5/10
1/1 [==============================] - 20s 20s/step - loss: 1.0420e-06 - accuracy: 0.3333
Epoch 6/10
1/1 [==============================] - 20s 20s/step - loss: 1.0163e-07 - accuracy: 0.3333
Epoch 7/10
1/1 [==============================] - 20s 20s/step - loss: 2.4848e-08 - accuracy: 0.3333
Epoch 8/10
1/1 [==============================] - 20s 20s/step - loss: 5.9447e-10 - accuracy: 0.3333
Epoch 9/10
1/1 [==============================] - 20s 20s/step - loss: 4.5714e-11 - accuracy: 0.3333
Epoch 10/10
1/1 [==============================] - 19s 19s/step - loss: 5.7787e-11 - accuracy: 0.3333


### Model predictions
As can been seen below, it correctly predicts that this video was a real one

In [ ]:
ind = 2
lab = model.predict([np.expand_dims(image_dataset[ind], axis=0), np.expand_dims(audio_dataset[ind], axis=0)])

if(lab == 0):
  print("FAKE")
else:
  print("REAL")

REAL
